<div style="text-align: right">   </div>


Introduction to Deep Learning (2024) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;| &nbsp;
-------|-------------------
**Assignment 2 - Sequence processing using RNNs** | <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/UniversiteitLeidenLogo.svg/1280px-UniversiteitLeidenLogo.svg.png" width="300">



# Introduction


The goal of this assignment is to learn how to use encoder-decoder recurrent neural networks (RNNs). Specifically we will be dealing with a sequence to sequence problem and try to build recurrent models that can learn the principles behind simple arithmetic operations (**integer addition, subtraction and multiplication.**).

<img src="https://i.ibb.co/5Ky5pbk/Screenshot-2023-11-10-at-07-51-21.png" alt="Screenshot-2023-11-10-at-07-51-21" border="0" width="500"></a>

In this assignment you will be working with three different kinds of models, based on input/output data modalities:
1. **Text-to-text**: given a text query containing two integers and an operand between them (+ or -) the model's output should be a sequence of integers that match the actual arithmetic result of this operation
2. **Image-to-text**: same as above, except the query is specified as a sequence of images containing individual digits and an operand.
3. **Text-to-image**: the query is specified in text format as in the text-to-text model, however the model's output should be a sequence of images corresponding to the correct result.


### Description**
Let us suppose that we want to develop a neural network that learns how to add or subtract
two integers that are at most two digits long. For example, given input strings of 5 characters: ‘81+24’ or
’41-89’ that consist of 2 two-digit long integers and an operand between them, the network should return a
sequence of 3 characters: ‘105 ’ or ’-48 ’ that represent the result of their respective queries. Additionally,
we want to build a model that generalizes well - if the network can extract the underlying principles behind
the ’+’ and ’-’ operands and associated operations, it should not need too many training examples to generate
valid answers to unseen queries. To represent such queries we need 13 unique characters: 10 for digits (0-9),
2 for the ’+’ and ’-’ operands and one for whitespaces ’ ’ used as padding.
The example above describes a text-to-text sequence mapping scenario. However, we can also use different
modalities of data to represent our queries or answers. For that purpose, the MNIST handwritten digit
dataset is going to be used again, however in a slightly different format. The functions below will be used to create our datasets.

---

*To work on this notebook you should create a copy of it.*


# Function definitions for creating the datasets

First we need to create our datasets that are going to be used for training our models.

In order to create image queries of simple arithmetic operations such as '15+13' or '42-10' we need to create images of '+' and '-' signs using ***open-cv*** library. We will use these operand signs together with the MNIST dataset to represent the digits.

In [1]:
# !pip install -r requirements.txt
# !pip install tensorflow[and-cuda]

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np
import random
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
import re
from sklearn.model_selection import train_test_split
from scipy.ndimage import rotate
from tensorflow.keras.layers import Conv2DTranspose, Dense, Reshape, LSTM, Dropout, TimeDistributed, RepeatVector, Input
from sklearn.metrics import recall_score, precision_score
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
# weet niet of dit bovenste stukje code nodig is, maar ik heb het er maar in gezet
tf.config.list_physical_devices('GPU')
# np.random.seed(42)
# tf.random.set_seed(42)
# random.seed(42)

In [ ]:
tf.__version__

In [ ]:
# Create plus/minus operand signs
def generate_images(number_of_images=50, sign='-'):
    blank_images = np.zeros([number_of_images, 28, 28])  # Dimensionality matches the size of MNIST images (28x28)
    x = np.random.randint(12, 16, (number_of_images, 2)) # Randomized x coordinates
    y1 = np.random.randint(6, 10, number_of_images)       # Randomized y coordinates
    y2 = np.random.randint(18, 22, number_of_images)     # -||-

    for i in range(number_of_images): # Generate n different images
        cv2.line(blank_images[i], (y1[i], x[i,0]), (y2[i], x[i, 1]), (255,0,0), 2, cv2.LINE_AA)     # Draw lines with randomized coordinates
        if sign == '+':
            cv2.line(blank_images[i], (x[i,0], y1[i]), (x[i, 1], y2[i]), (255,0,0), 2, cv2.LINE_AA) # Draw lines with randomized coordinates
        if sign == '*':
            cv2.line(blank_images[i], (x[i,0], y1[i]), (x[i, 1], y2[i]), (255,0,0), 2, cv2.LINE_AA)
            # Rotate 45 degrees
            blank_images[i] = rotate(blank_images[i], -50, reshape=False)
            cv2.line(blank_images[i], (x[i,0], y1[i]), (x[i, 1], y2[i]), (255,0,0), 2, cv2.LINE_AA)
            blank_images[i] = rotate(blank_images[i], -50, reshape=False)
            cv2.line(blank_images[i], (x[i,0], y1[i]), (x[i, 1], y2[i]), (255,0,0), 2, cv2.LINE_AA)

    return blank_images

def show_generated(images, n=5):
    plt.figure(figsize=(2, 2))
    for i in range(n**2):
        plt.subplot(n, n, i+1)
        plt.axis('off')
        plt.imshow(images[i])
    plt.show()

show_generated(generate_images(sign='-'))
show_generated(generate_images(sign='+'))
show_generated(generate_images(sign='*'))

In [5]:
def create_data(highest_integer, num_addends=2, operands=['+', '-']):
    """
    Creates the following data for all pairs of integers up to [1:highest integer][+/-][1:highest_integer]:

    @return:
    X_text: '51+21' -> text query of an arithmetic operation (5)
    X_img : Stack of MNIST images corresponding to the query (5 x 28 x 28) -> sequence of 5 images of size 28x28
    y_text: '72' -> answer of the arithmetic text query
    y_img :  Stack of MNIST images corresponding to the answer (3 x 28 x 28)

    Images for digits are picked randomly from the whole MNIST dataset.
    """

    num_indices = [np.where(MNIST_labels==x) for x in range(10)]
    num_data = [MNIST_data[inds] for inds in num_indices]
    image_mapping = dict(zip(unique_characters[:10], num_data))
    image_mapping['-'] = generate_images()
    image_mapping['+'] = generate_images(sign='+')
    image_mapping['*'] = generate_images(sign='*')
    image_mapping[' '] = np.zeros([1, 28, 28])

    X_text, X_img, y_text, y_img = [], [], [], []

    for i in range(highest_integer + 1):      # First addend
        for j in range(highest_integer + 1):  # Second addend
            for sign in operands: # Create all possible combinations of operands
                query_string = to_padded_chars(str(i) + sign + str(j), max_len=max_query_length, pad_right=True)
                query_image = []
                for n, char in enumerate(query_string):
                    image_set = image_mapping[char]
                    index = np.random.randint(0, len(image_set), 1)
                    query_image.append(image_set[index].squeeze())

                result = eval(query_string)
                result_string = to_padded_chars(result, max_len=max_answer_length, pad_right=True)
                result_image = []
                for n, char in enumerate(result_string):
                    image_set = image_mapping[char]
                    index = np.random.randint(0, len(image_set), 1)
                    result_image.append(image_set[index].squeeze())

                X_text.append(query_string)
                X_img.append(np.stack(query_image))
                y_text.append(result_string)
                y_img.append(np.stack(result_image))

    # Combine the two lists into a single list of tuples
    combined_text = list(zip(X_text, y_text))
    combined_img = list(zip(X_img, y_img))
    combined = list(zip(combined_text, combined_img))
    random.shuffle(combined)
    # unpack combined
    X_text, y_text = zip(*[x for x, _ in combined])
    X_img, y_img = zip(*[x for _, x in combined])
    return np.stack(X_text), np.stack(X_img)/255., np.stack(y_text), np.stack(y_img)/255.

def to_padded_chars(integer, max_len=3, pad_right=False):
    """
    Returns a string of len()=max_len, containing the integer padded with ' ' on either right or left side
    """
    length = len(str(integer))
    padding = (max_len - length) * ' '
    if pad_right:
        return str(integer) + padding
    else:
        return padding + str(integer)


# Creating our data

The dataset consists of 20000 samples that (additions and subtractions between all 2-digit integers) and they have two kinds of inputs and label modalities:

  **X_text**: strings containing queries of length 5: ['  1+1  ', '11-18', ...]

  **X_image**: a stack of images representing a single query, dimensions: [5, 28, 28]

  **y_text**: strings containing answers of length 3: ['  2', '156']

  **y_image**: a stack of images that represents the answer to a query, dimensions: [3, 28, 28]

In [ ]:
# Illustrate the generated query/answer pairs

unique_characters = '0123456789+- '       # All unique characters that are used in the queries (13 in total: digits 0-9, 2 operands [+, -], and a space character ' '.)
highest_integer = 99                      # Highest value of integers contained in the queries

max_int_length = len(str(highest_integer))# Maximum number of characters in an integer
max_query_length = max_int_length * 2 + 1 # Maximum length of the query string (consists of two integers and an operand [e.g. '22+10'])
max_answer_length = 3    # Maximum length of the answer string (the longest resulting query string is ' 1-99'='-98')

# Create the data (might take around a minute)
(MNIST_data, MNIST_labels), _ = tf.keras.datasets.mnist.load_data()
X_text, X_img, y_text, y_img = create_data(highest_integer)
## Display the samples that were created
def display_sample(n):
    labels = ['X_img:', 'y_img:']
    for i, data in enumerate([X_img, y_img]):
        plt.subplot(1,2,i+1)
        # plt.set_figheight(15)
        plt.axis('off')
        plt.title(labels[i])
        plt.imshow(np.hstack(data[n]), cmap='gray')
    print('='*50, f'\nQuery #{n}\n\nX_text: "{X_text[n]}" = y_text: "{y_text[n]}"')
    plt.show()

for _ in range(10):
    display_sample(np.random.randint(0, 10000, 1)[0])

## Helper functions

The functions below will help with input/output of the data.

In [ ]:
# One-hot encoding/decoding the text queries/answers so that they can be processed using RNNs
# You should use these functions to convert your strings and read out the output of your networks
def encode_labels(labels, max_len=3):
  n = len(labels)
  length = len(labels[0])
  char_map = dict(zip(unique_characters, range(len(unique_characters))))
  one_hot = np.zeros([n, length, len(unique_characters)])
  for i, label in enumerate(labels):
      m = np.zeros([length, len(unique_characters)])
      for j, char in enumerate(label):
          m[j, char_map[char]] = 1
      one_hot[i] = m

  return one_hot


def decode_labels(labels):
    pred = np.argmax(labels, axis=1)
    predicted = ''.join([unique_characters[i] for i in pred])
    return predicted
# TODO set unrandom but for debug its fine
X_text_onehot = encode_labels(X_text)
y_text_onehot = encode_labels(y_text)
random_numbers = np.random.randint(0, len(X_text), 4)
print(X_text_onehot.shape, y_text_onehot.shape)
for i in range(4):
    print(y_text_onehot[random_numbers[i]])
    print(decode_labels(y_text_onehot[random_numbers[i]]))

In [8]:
def check_classification(y_pred, y_true):
  bools = [y_pred[i] == y_true[i] for i in range(len(y_pred))]
  correct = np.where(bools)[0]
  incorrect = np.where(np.logical_not(bools))[0]
  return len(correct), len(incorrect), np.array(correct), np.array(incorrect, dtype=int)

def decode_all(predictions):
    decoded_predictions = []
    for i in range(len(predictions)):
        decoded_predictions.append(decode_labels(predictions[i]))
    return decoded_predictions

In [ ]:
# we need to convert our strings to integers. 
# Note that due to white spacing we must first subtract any white spacing, before converting to integers. 
# Furthermore, we should note exceptions such as '- ' which is basically just a black, we should completely remove these, but do count them up. 
# These are of course completely wrong as no number is present.

def process_strings(input_list):
    cleaned_list = []
    invalid_count = 0

    for item in input_list:
        cleaned_item = item.replace(" ", "")
        # # check if there is only a + or - in the string
        if re.search(r"[+-]", cleaned_item[1:]) or re.search(r"[^0-9+-]", cleaned_item) or not cleaned_item or cleaned_item == '-':
            invalid_count += 1
            continue


        cleaned_item_int = int(cleaned_item)
        cleaned_list.append(cleaned_item_int)
    return cleaned_list, invalid_count

# for debugging:
input_strings = [" 92 ", "30", "-67", "  -  20 ", "12 - 34", "49-", '']
cleaned_numbers, invalid_count = process_strings(input_strings)

print("Cleaned Numbers:", cleaned_numbers)
print("Invalid Count:", invalid_count)

In [10]:
# visualize the wrongly classified instances (2.2)
# Set the batch size

def plot_wrong_classified(cleaned_wrong_classified, cleaned_true,batch_size=300):

    # Get the total number of instances
    total_instances = len(cleaned_wrong_classified)

    # Calculate the number of subplots needed
    num_subplots = int(np.ceil(total_instances / batch_size))

    # Calculate the number of rows and columns for the subplot grid
    num_rows = int(np.sqrt(num_subplots))
    num_cols = int(np.ceil(num_subplots / num_rows))

    # Create a subplot grid
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(24, 12))

    # Flatten the axs array if it's a multi-dimensional array
    axs = axs.flatten()

    # Plot instances in batches of 250
    for i in range(num_subplots):
        start_index = i * batch_size
        end_index = min((i + 1) * batch_size, total_instances)

        # Use the current subplot
        ax = axs[i]

        # Plot instances that were wrongly labeled
        ax.scatter(range(start_index, end_index), cleaned_wrong_classified[start_index:end_index],
                color='red', marker='x', label='Wrong predicted values', s=25)

        # Plot true values of the wrongly classified instances
        ax.scatter(range(start_index, end_index), cleaned_true[start_index:end_index],
                color='blue', marker='o', label='True values', s=25, alpha=0.4)

        # Customize the subplot
        ax.set_xlabel('Instance index')
        ax.set_ylabel('Instance value')
        ax.set_title(f'Subplot {i+1}')
        ax.grid()

        # Create a legend with unique labels
        ax.legend()

    # Adjust layout to prevent clipping of titles
    plt.tight_layout()
    # Display the grid of subplots
    plt.show()

---
---

## I. Text-to-text RNN model

The following code showcases how Recurrent Neural Networks (RNNs) are built using Keras. Several new layers are going to be used:

1. LSTM
2. TimeDistributed
3. RepeatVector

The code cell below explains each of these new components.

<img src="https://i.ibb.co/NY7FFTc/Screenshot-2023-11-10-at-09-27-25.png" alt="Screenshot-2023-11-10-at-09-27-25" border="0" width="500"></a>


In [11]:
def build_text2text_model(num_encode_layers=1,num_decode_layers=1,max_answer_length=3):
    text2text = tf.keras.Sequential()
    text2text.add(Input(shape=(max_query_length, len(unique_characters))))
    hidden_units = 128
    for i in range(num_encode_layers):
        text2text.add(LSTM(hidden_units, return_sequences=(i < num_encode_layers - 1)))
        text2text.add(Dropout(0.2))
    # Repeat the context vector for decoding
    text2text.add(RepeatVector(max_answer_length))

    # Decoder: Add num_decode_layers LSTM layers
    for _ in range(num_decode_layers):
        text2text.add(LSTM(hidden_units, return_sequences=True))
        text2text.add(Dropout(0.2))

    # Output layer: TimeDistributed Dense with softmax activation
    text2text.add(TimeDistributed(Dense(len(unique_characters), activation='softmax')))


    # Next we compile the model using categorical crossentropy as our loss function.
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    text2text.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return text2text

In [ ]:
model = build_text2text_model()

In [ ]:
training_sizes = np.array([i/10 for i in range(1, 10)])
training_sizes

In [ ]:
## Your code (look at the assignment description for your tasks for text-to-text model):
## Your first task is to fit the text2text model using X_text and y_text)
## dit is taak 1.2
test_split = 0.1
X_test = X_text_onehot[:int(len(X_text_onehot)*test_split)]
y_test = y_text_onehot[:int(len(y_text_onehot)*test_split)]

X_train = X_text_onehot[int(len(X_text_onehot)*test_split):]
y_train = y_text_onehot[int(len(y_text_onehot)*test_split):]
total_scores = {}
batch_size = 128
epochs = 10
for train_size in training_sizes[:]:
    print(f"Training size: {train_size}, Test split: {test_split}, validation size: {1-train_size}")

    X_train_batch, X_val_batch,y_train_batch, y_val_batch = train_test_split(X_train, y_train, train_size=train_size, random_state=42)
    model = build_text2text_model()
    history = model.fit(X_train_batch, y_train_batch, validation_data=(X_val_batch, y_val_batch), epochs=epochs, batch_size=batch_size)
    # history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=128)
    results = model.evaluate(X_test, y_test)
    results = {'loss': results[0], 'accuracy': results[1], 'history': history}
    print(f"Test {results}")
    total_scores[train_size] = results

In [ ]:
# verslag table
for key,value in total_scores.items():
    sentence = f"{round(1-key,2)} & {round(value['accuracy']*100,2)} & {round(value['loss'],4)} \\\\"
    print(sentence)

In [ ]:
best_config_key = max(total_scores, key=lambda x: total_scores[x]['accuracy'])
best_config = total_scores[best_config_key]
best_config_key,best_config

In [ ]:
X_train_batch, X_val_batch,y_train_batch, y_val_batch = train_test_split(X_train, y_train, train_size=best_config_key, random_state=42)
model = build_text2text_model()
history = model.fit(X_train_batch, y_train_batch, validation_data=(X_val_batch, y_val_batch),epochs=epochs, batch_size=batch_size)

In [18]:
import pickle
with open('total_scores.pkl', 'wb') as f:
    pickle.dump(total_scores, f)

In [ ]:
def create_history_plots(histories, sizes):
    plt.figure(figsize=(10, 10))
    for i, history in enumerate(histories):
        plt.subplot(3, 3, i+1)
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        # set title fontsize a bit lower
        plt.title(f'Accuracy with {int(sizes[i]*100)}% training size', fontsize=8)
        plt.grid()
        plt.ylim(0,1)
        plt.ylabel('accuracy', fontsize=8)
        plt.xlabel('epoch', fontsize=8)
        plt.legend(['train', 'val'], loc='upper left')
        # set x and y values also to fontsize 8
        plt.xticks(fontsize=8)
        plt.yticks(fontsize=8)
    plt.show()


histories = [total_scores[key]['history'] for key in total_scores.keys()]
sizes = [key for key in total_scores.keys()]
create_history_plots(histories, sizes)

In [ ]:
predictions = model.predict(X_test)
# put them into a readable format
for i in range(len(predictions)):
    print(f"Predicted: {decode_labels(predictions[i])}, True: {decode_labels(y_test[i])}")

In [ ]:
decoded_predictions = decode_all(predictions)
decoded_y_test = decode_all(y_test)
correct_num, incorrect_num, correct_indices, incorrect_indices = check_classification(decoded_predictions, decoded_y_test)
print(f"Correct: {correct_num}, Incorrect: {incorrect_num}")
print(f"accuracy: {correct_num/(correct_num+incorrect_num)}")

In [ ]:
cleaned_wrong_classified, invalid_count_wrong_classified = process_strings(np.array(decoded_predictions)[incorrect_indices])
cleaned_true, invalid_count_true = process_strings(np.array(decoded_y_test)[incorrect_indices])
print(invalid_count_wrong_classified, invalid_count_true)
int_decoded_predictions = [int(x) for x in cleaned_wrong_classified]
int_decoded_y_test = [int(x) for x in decoded_y_test]
plt.hist(int_decoded_predictions, bins=range(-100,100), alpha=0.5, label='Predictions')
plt.hist(int_decoded_y_test, bins=range(-100,100), alpha=0.5, label='True')
plt.legend(loc='upper right')
plt.xlabel('Number')
plt.ylabel('Amount')
plt.title('Distribution of correct and incorrect numbers')
# logarithm y-axis
plt.yscale('log')
plt.show()

In [ ]:
plot_wrong_classified(cleaned_wrong_classified, cleaned_true)


adding multiple layers

In [ ]:
best_config_key,best_config

In [ ]:
def test_num_decode_encode(train_size,num_of_decodes = 1,
                           num_of_encodes = 1):
  # Create datasets
  X_train_batch, X_val_batch,y_train_batch, y_val_batch = train_test_split(X_train, y_train, train_size=train_size, random_state=42)
  results_all = {}
  # Iterate over encoder en decoder numbers
  for i in range(1,num_of_decodes+1):
      for j in range(1,num_of_encodes+1):
        print(f"Testing {i} decode, {j} encode layers")
        model = build_text2text_model(num_encode_layers=j, num_decode_layers=i)
        history = model.fit(X_train_batch, y_train_batch, validation_data=(X_val_batch, y_val_batch),epochs=10, batch_size=128)
        results_all[(i,j)] = {'history': history}
  return results_all
results = test_num_decode_encode(num_of_decodes = 1,
                       num_of_encodes = 5,train_size=best_config_key)

results2 = test_num_decode_encode(num_of_decodes = 5,
                        num_of_encodes = 1,train_size=best_config_key)

In [ ]:
results2

In [ ]:
results

In [ ]:
# combine the results
results_comb = {**results, **results2}
results_comb

In [ ]:
for key, value in results_comb.items():
    print(key, value['history'].history['val_accuracy'][-1])

In [ ]:
best_config = max(results_comb, key=lambda k: results_comb[k]['history'].history['val_accuracy'][-1])
best_config

In [ ]:
# train new cool model
X_train_batch, X_val_batch,y_train_batch, y_val_batch = train_test_split(X_train, y_train, train_size=train_size, random_state=42)
model = build_text2text_model(num_encode_layers=best_config[1], num_decode_layers=best_config[0])
history = model.fit(X_train_batch, y_train_batch, validation_data=(X_val_batch, y_val_batch),epochs=10, batch_size=128)
predictions = model.predict(X_test)

In [ ]:
decoded_predictions = decode_all(predictions)
decoded_y_test = decode_all(y_test)
cleaned_wrong_classified, invalid_count_wrong_classified = process_strings(np.array(decoded_predictions)[incorrect_indices])
cleaned_true, invalid_count_true = process_strings(np.array(decoded_y_test)[incorrect_indices])
print(f"accuracy: {correct_num/(correct_num+incorrect_num)}")
plot_wrong_classified(cleaned_wrong_classified, cleaned_true)


---
---

## II. Image to text RNN Model

Hint: There are two ways of building the encoder for such a model - again by using the regular LSTM cells (with flattened images as input vectors) or recurrect convolutional layers [ConvLSTM2D](https://keras.io/api/layers/recurrent_layers/conv_lstm2d/).

The goal here is to use **X_img** as inputs and **y_text** as outputs.

In [48]:
def build_img2text_model(num_encode_layers: int = 1,num_decode_layers:int = 1, max_answer_length: int=3):
    img2text = tf.keras.Sequential()
    img2text.add(Input(shape=(5, 28*28)))

    hidden_units = 256
    # Encoder: Add num_encode_layers LSTM layers
    for _ in range(num_encode_layers):
        img2text.add(LSTM(hidden_units, return_sequences=False if _ == num_encode_layers - 1 else True))

    # Repeat the context vector for decoding
    img2text.add(RepeatVector(max_answer_length))

    # Decoder: Add num_decode_layers LSTM layers
    for i in range(num_decode_layers):
        img2text.add(LSTM(hidden_units, return_sequences=True))

    # Output layer: Dense layer with softmax activation for classification
    img2text.add(Dense(len(unique_characters), activation='softmax'))

    # Compile the model
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    img2text.compile(loss='categorical_crossentropy', optimizer=opt,
                     metrics=['accuracy'], steps_per_execution=25)
    return img2text

In [ ]:
X_img_reshaped = X_img.reshape(-1, 5, 28 * 28)
test_size = 0.1
X_train = X_img_reshaped[int(len(X_img_reshaped)*test_size):]
y_train = y_text_onehot[int(len(y_text_onehot)*test_size):]
X_test = X_img_reshaped[:int(len(X_img_reshaped)*test_size)]
y_test = y_text_onehot[:int(len(y_text_onehot)*test_size)]


X_train_batch, X_val_batch, y_train_batch, y_val_batch = train_test_split(X_train , y_train, train_size=best_config_key, random_state=42)
epochs = 10
model = build_img2text_model(num_encode_layers=best_config[1], num_decode_layers=best_config[0])
history = model.fit(X_train,y_train,batch_size=32,validation_data= (X_val_batch, y_val_batch), epochs=epochs)

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']
print("Result:")
model.evaluate(X_test,y_test)

plt.plot(np.arange(1, epochs+1), acc, label='Training', color='blue')
plt.grid()
plt.ylim(0, 1)
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [ ]:
# import recall, precision
predictions = model.predict(X_test)
decoded_predictions = decode_all(predictions)
decoded_y_test = decode_all(y_test)
correct_num, incorrect_num, correct_indices, incorrect_indices = check_classification(decoded_predictions, decoded_y_test)
print(f"Correct: {correct_num}, Incorrect: {incorrect_num}")
print(f"accuracy: {correct_num/(correct_num+incorrect_num)}")
test_recall = recall_score(decoded_y_test, decoded_predictions,average='weighted')
test_precision = precision_score(decoded_y_test, decoded_predictions,average='weighted')
print(f"Recall: {test_recall}, Precision: {test_precision}, Accuracy: {correct_num/(correct_num+incorrect_num)}")

cleaned_wrong_classified, invalid_count_wrong_classified = process_strings(np.array(decoded_predictions)[incorrect_indices])
cleaned_true, invalid_count_true = process_strings(np.array(decoded_y_test)[incorrect_indices])
print(invalid_count_wrong_classified, invalid_count_true)
print(f"accuracy: {correct_num/(correct_num+incorrect_num)}")
plot_wrong_classified(cleaned_wrong_classified, cleaned_true)

In [ ]:
# 1.5
def test_num_decode_encode_img2text(train_size,num_of_decodes = [1,2,3,4,5],
                           num_of_encodes = [1,2,3,4,5]):
  # Create datasets
  X_train_batch, X_val_batch,y_train_batch, y_val_batch = train_test_split(X_train, y_train, train_size=train_size, random_state=42)
  results_all = {}
  # Iterate over encoder en decoder numbers
  for i in num_of_decodes:
    for j in num_of_encodes:
        print(f"Testing {i} decode, {j} encode layers")
        model = build_img2text_model(num_encode_layers=j, num_decode_layers=i)
        history = model.fit(X_train_batch, y_train_batch, validation_data=(X_val_batch, y_val_batch),epochs=10, batch_size=128)
        results_all[(i,j)] = {'history': history}
  return results_all
results_imgtext = test_num_decode_encode_img2text(num_of_decodes = [1,2,3,4,5],
                       num_of_encodes = [1],train_size=0.9) # TODO change naar best config key
results2_imgtext = test_num_decode_encode_img2text(num_of_decodes = [1],
                       num_of_encodes = [1,2,3,4,5],train_size=0.9) # TODO change naar best config key
results_comb_imgtext = {**results_imgtext, **results2_imgtext}
best_config_imgtext = max(results_comb_imgtext, key=lambda k: results_comb_imgtext[k]['history'].history['val_accuracy'][-1])
for key, value in results_comb_imgtext.items():
    print(key, value['history'].history['val_accuracy'][-1])
X_train_batch, X_val_batch,y_train_batch, y_val_batch = train_test_split(X_train, y_train, train_size=0.9, random_state=42)
model = build_img2text_model(num_encode_layers=best_config_imgtext[1], num_decode_layers=best_config_imgtext[0])
history = model.fit(X_train_batch, y_train_batch, validation_data=(X_val_batch, y_val_batch),epochs=10, batch_size=128)
predictions = model.predict(X_test)
decoded_predictions = decode_all(predictions)
decoded_y_test = decode_all(y_test)
cleaned_wrong_classified, invalid_count_wrong_classified = process_strings(np.array(decoded_predictions)[incorrect_indices])
cleaned_true, invalid_count_true = process_strings(np.array(decoded_y_test)[incorrect_indices])
print(f"accuracy: {correct_num/(correct_num+incorrect_num)}")
plot_wrong_classified(cleaned_wrong_classified, cleaned_true)

---
---

## III. Text to image RNN Model

Hint: to make this model work really well you could use deconvolutional layers in your decoder (you might need to look up ***Conv2DTranspose*** layer). However, regular vector-based decoder will work as well.

The goal here is to use **X_text** as inputs and **y_img** as outputs.

In [ ]:
# Your code
def build_text2img_model(num_encode_layers=1, num_decode_layers=1, max_answer_length=5):
    model = tf.keras.Sequential()
    
    # Encoder: Dynamically add LSTM layers
    for i in range(num_encode_layers):
        return_sequences = (i < num_encode_layers - 1)  # Only the last encoder layer doesn't return sequences
        model.add(LSTM(512, return_sequences=return_sequences, input_shape=(3, 13), name=f"encoder_lstm_{i + 1}"))
        model.add(Dropout(0.4, name=f"encoder_dropout_{i + 1}"))
    
    # Repeat vector to match the number of outputs (5 images in a sequence)
    model.add(RepeatVector(max_answer_length, name="repeat_vector"))  # (batch_size, max_answer_length, 512)
    
    # Decoder: Dynamically add LSTM layers
    for i in range(num_decode_layers):
        model.add(LSTM(256 if i == 0 else 128, return_sequences=True, name=f"decoder_lstm_{i + 1}"))
        model.add(Dropout(0.4, name=f"decoder_dropout_{i + 1}"))
    
    # Generate the spatial features for each time step
    model.add(TimeDistributed(Dense(7 * 7 * 128, activation='relu'), name="time_dense"))
    model.add(TimeDistributed(Reshape((7, 7, 128)), name="time_reshape"))
    
    # Transpose convolutions to upscale to (28, 28)
    model.add(TimeDistributed(Conv2DTranspose(128, (3, 3), strides=2, padding='same', activation='relu'), name="conv2d_transpose_1"))
    model.add(TimeDistributed(Conv2DTranspose(64, (3, 3), strides=2, padding='same', activation='relu'), name="conv2d_transpose_2"))
    model.add(TimeDistributed(Conv2DTranspose(32, (3, 3), padding='same', activation='relu'), name="conv2d_transpose_3"))
    model.add(TimeDistributed(Conv2DTranspose(1, (3, 3), padding='same', activation='sigmoid'), name="output"))
    
    # Compile the model
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    
    return model

model = build_text2img_model()
model.summary()

In [ ]:
X_img_reshaped = X_img.reshape(-1, 5, 28,28)
test_size = 0.1
y_train = X_img_reshaped[int(len(X_img_reshaped)*test_size):]
X_train = y_text_onehot[int(len(y_text_onehot)*test_size):]
y_test = X_img_reshaped[:int(len(X_img_reshaped)*test_size)]
X_test = y_text_onehot[:int(len(y_text_onehot)*test_size)]
best_config_key = 0.9
X_train_batch, X_val_batch, y_train_batch, y_val_batch = train_test_split(X_train , y_train, train_size=best_config_key, random_state=42)
epochs = 10
model = build_text2img_model(max_answer_length=5)
history = model.fit(X_train,y_train,batch_size=128,validation_data= (X_val_batch, y_val_batch), epochs=epochs)
predictions = model.predict(X_test)


In [ ]:
def display_sample_prediction(n):
    labels = ['y_pred:', 'y_test:']
    for j, data in enumerate([predictions, y_test]):
        plt.subplot(1,2,j+1)
        plt.axis('off')
        plt.title(labels[j])
        plt.imshow(np.hstack(data[n]), cmap='gray')
    plt.show()

print(predictions.shape)
for i in np.random.randint(0,predictions.shape[0],5):
  display_sample_prediction(i)

In [ ]:
def test_num_decode_encode_text2img(train_size,num_of_decodes = [1,2,3,4,5],
                           num_of_encodes = [1,2,3,4,5]):
  # Create datasets
  X_train_batch, X_val_batch,y_train_batch, y_val_batch = train_test_split(X_train, y_train, train_size=train_size, random_state=42)
  results_all = {}
  # Iterate over encoder en decoder numbers
  for i in num_of_decodes:
    for j in num_of_encodes:
        print(f"Testing {i} decode, {j} encode layers")
        model = build_text2img_model(num_encode_layers=j, num_decode_layers=i,max_answer_length=5)
        history = model.fit(X_train_batch, y_train_batch, validation_data=(X_val_batch, y_val_batch),epochs=10, batch_size=128)
        results_all[(i,j)] = {'history': history}
  return results_all

results_text2img= test_num_decode_encode_text2img(num_of_decodes = [1,2,3,4,5],
                       num_of_encodes = [1],train_size=0.9) # TODO change naar best config key


In [ ]:
results2_text2img = test_num_decode_encode_text2img(num_of_decodes = [1],
                       num_of_encodes = [1,2,3,4,5],train_size=0.9) # TODO change naar best config key
results_comb_text2img = {**results_text2img, **results2_text2img}
best_config_text2img = max(results_comb_text2img, key=lambda k: results_comb_text2img[k]['history'].history['val_mse'][-1])
for key, value in results_comb_text2img.items():
    print(key, value['history'].history['val_mse'][-1])


In [ ]:
X_train_batch, X_val_batch,y_train_batch, y_val_batch = train_test_split(X_train, y_train, train_size=0.9, random_state=42)
model = build_text2img_model(num_encode_layers=best_config_text2img[1], num_decode_layers=best_config_text2img[0])
history = model.fit(X_train_batch, y_train_batch, validation_data=(X_val_batch, y_val_batch),epochs=10, batch_size=128)
predictions = model.predict(X_test)


In [ ]:
def display_sample_prediction(n):
    labels = ['y_pred:', 'y_test:']
    for j, data in enumerate([predictions, y_test]):
        plt.subplot(1,2,j+1)
        plt.axis('off')
        plt.title(labels[j])
        plt.imshow(np.hstack(data[n]), cmap='gray')
    plt.show()

print(predictions.shape)
for i in np.random.randint(0,predictions.shape[0],5):
  display_sample_prediction(i)

In [ ]:
mse = model.evaluate(X_test,y_test)
mse